Aplicando restrições na busca
=============================



## Introdução



Muitos problemas de otimização com relevância científica têm uma ou mais `restrições` que devem ser levadas em consideração na hora de resolver o problema.

Lembra do `problema da mochila` que vimos em Lógica Computacional? Era um problema de otimização onde queríamos maximizar o valor dos itens colocados na mochila enquanto observávamos a restrição do peso total dos itens (do contrário, a mochila rasgava).

Uma forma de considerar essas restrições nos problemas é aplicando uma `penalidade` na função objetivo.

Vamos pensar como seria essa penalidade no problema da mochila: a função objetivo é maximizar o valor dos itens na mochila, então é um problema de maximização. A função objetivo pode ser a soma dos itens da mochila. Se fosse só isso, teríamos

$$
f = \sum_{i, i \in \mathrm{mochila}}\mathrm{valor}(i)
$$

No entanto, apenas essa função não resolve o problema! Precisamos levar em consideração o limite de peso da mochila! Para isso, penalizamos a função objetivo levando em consideração essa restrição:

$f=\begin{cases}
0.01 & \textrm{se peso > limite da mochila}\\
\sum_{i,i\in\mathrm{mochila}}(\mathrm{valor}(i)) & \textrm{se peso} \leq \textrm{limite da mochila}
\end{cases}$

Agora finalmente podemos seguir em frente e resolver o problema.



## Reflexões



Se usarmos a equação de $f$ acima, qual será o valor de $f$ caso não exista uma solução para um certo problema da mochila?

Na equação de $f$ acima nós usamos um valor práximo de zero para indicar que uma restrição do problema não foi satisfeita. Você consegue pensar em outra estratégia para penalizar soluções inválidas?



## Objetivo



Encontrar uma solução para o problema da mochila usando algoritmos genéticos. Considere que existem 10 itens diferentes (com pesos e valores diferentes) disponíveis para serem escolhidos.



## Descrição do problema



No problema da mochila você tem um número $n$ de itens disponíveis, cada um com um peso e um valor associado. Sua mochila tem a capacidade de carregar um número $p$ de quilogramas, sendo que mais que isso faz com que sua mochila rasgue e todos os itens dentro dela caiam no chão e se quebrem de maneira catastrófica (indesejado). Sua tarefa é encontrar um conjunto de itens (considerando os $n$ disponíveis) que maximize o valor contido dentro da mochila, porém que tenham um peso dentro da capacidade da mesma.



## Importações



In [1]:
import random

from funcoes import computa_mochila
from funcoes import funcao_objetivo_pop_mochila
from funcoes import populacao_cb as cria_populacao_inicial
from funcoes import selecao_roleta_max as funcao_selecao
from funcoes import cruzamento_ponto_simples as funcao_cruzamento
from funcoes import mutacao_cb as funcao_mutacao

A primeira etapa do código consiste em importar as funções necessárias para implementação do algoritmo e para seu funcionamento. Além disto, é importada a biblioteca random que também é necessária para o cálculo de probabilidade de cruzamento e mutação

## Códigos e discussão



In [2]:
### CONSTANTES

# relacionadas à busca
TAMANHO_POP = 20
NUM_GERACOES = 10
CHANCE_CRUZAMENTO = 0.5
CHANCE_MUTACAO = 0.05

# relacionadas ao problema a ser resolvido
LIMITE_DE_PESO = 15
OBJETOS = {
    # dicionário baseado no que vocês enviaram na aula de Lógica

    "vinil falsificado da volta do One Direction": {
        "peso": 2,
        "valor": 2500,
    },
    "Harry Potter: ele voltou, confia!": {
        "peso": 3,
        "valor": 1500,
    },
    "Quadrinho super raro do Aranha-Homem da vida real": {
        "peso": 3,
        "valor": 7000,
    },
    "mesa dobrável para laptop": {
        "peso": 3,
        "valor": 150,
    },
    "tablet": {
        "peso": 0.6,
        "valor": 2400,
    },
    "teclado musical": {
        "peso": 3.5,
        "valor": 3000,
    },
    "bicicleta": {
        "peso": 16,
        "valor": 1000,
    },
    "lições em dia": {
        "peso": 8,
        "valor": 5000,
    },
    "energético": {
        "peso": 2,
        "valor": 1500,
    },
    "docinhos para o stress": {
        "peso": 5,
        "valor": 3000,
    },
}
NUM_OBJETOS = len(OBJETOS)
ORDEM_DOS_NOMES = list(sorted(OBJETOS.keys()))

A proxima etapa consiste em definir as constantes da busca, sendo estas estritamente relacionadas ao modo que o algoritmo será rodado, e também a importação das constantes do problema, estas, analogas às outras, estando estritamente relacionadas ao problema em si, e não ao algoritmo.

In [3]:
# Funções locais

def funcao_objetivo_pop(populacao):
    return funcao_objetivo_pop_mochila(
        populacao, OBJETOS, LIMITE_DE_PESO, ORDEM_DOS_NOMES
    )

Nesta etapa é feita a definição de funções locais, que são encarregadas passar variáveis goblais utilizadas por função que o algoritmo não acessa diretamente em sua função principal.

In [4]:
def algoritmo_completo(TAMANHO_POP, NUM_OBJETOS, NUM_GERACOES, CHANCE_CRUZAMENTO, CHANCE_MUTACAO, ORDEM_DOS_NOMES):
    # Busca por algoritmo genético

    populacao = cria_populacao_inicial(TAMANHO_POP, NUM_OBJETOS)

    # variaveis para o hall da fama
    melhor_fitness_ja_visto = -float("inf")
    melhor_individuo_ja_visto = [0] * NUM_OBJETOS

    for n in range(NUM_GERACOES):

        # Seleção
        fitness = funcao_objetivo_pop(populacao)
        populacao = funcao_selecao(populacao, fitness)

        # Cruzamento
        pais = populacao[0::2]
        maes = populacao[1::2]

        contador = 0

        for pai, mae in zip(pais, maes):
            if random.random() <= CHANCE_CRUZAMENTO:
                filho1, filho2 = funcao_cruzamento(pai, mae)
                populacao[contador] = filho1
                populacao[contador + 1] = filho2

            contador = contador + 2

        # Mutação
        for n in range(len(populacao)):
            if random.random() <= CHANCE_MUTACAO:
                individuo = populacao[n]
                populacao[n] = funcao_mutacao(individuo)

        # melhor individuo já visto até agora (hall da fama)
        fitness = funcao_objetivo_pop(populacao)
        maior_fitness = max(fitness)
        posicao = fitness.index(maior_fitness)
        individuo = populacao[posicao].copy()
        valor, peso = computa_mochila(individuo, OBJETOS, ORDEM_DOS_NOMES)
        if maior_fitness > melhor_fitness_ja_visto and peso <= LIMITE_DE_PESO:
            melhor_fitness_ja_visto = maior_fitness
            melhor_individuo_ja_visto = individuo
            print(f"Maior valor: {valor} | Peso: {peso}")


    # reportando o melhor individuo encontrado
    print()
    print("Você deve pegar os seguintes itens:")
    for pega_ou_nao, item in zip(melhor_individuo_ja_visto, ORDEM_DOS_NOMES):
        if pega_ou_nao == 1:
            print("+", item)
    print()
    valor_total, peso_total = computa_mochila(
        melhor_individuo_ja_visto, OBJETOS, ORDEM_DOS_NOMES
    )
    print(
        f"Com isso, sua mochila terá o valor de {valor_total} dinheiros "
        f"e peso de {peso_total} unidades de massa."
    )

for i, n in enumerate(range(10, 101, 10)):
    print(f'{i + 1}ª tentativa: {n} gerações:')
    NUM_GERACOES = n
    algoritmo_completo(TAMANHO_POP, NUM_OBJETOS, NUM_GERACOES, CHANCE_CRUZAMENTO, CHANCE_MUTACAO, ORDEM_DOS_NOMES)
    print('\n===========================================================================================================================================\n')

1ª tentativa: 10 gerações:
Maior valor: 12500 | Peso: 10
Maior valor: 14000 | Peso: 13
Maior valor: 15400 | Peso: 13.6
Maior valor: 15500 | Peso: 15
Maior valor: 16400 | Peso: 12.6

Você deve pegar os seguintes itens:
+ Quadrinho super raro do Aranha-Homem da vida real
+ docinhos para o stress
+ energético
+ tablet
+ vinil falsificado da volta do One Direction

Com isso, sua mochila terá o valor de 16400 dinheiros e peso de 12.6 unidades de massa.


2ª tentativa: 20 gerações:
Maior valor: 17900 | Peso: 14.1

Você deve pegar os seguintes itens:
+ Quadrinho super raro do Aranha-Homem da vida real
+ docinhos para o stress
+ tablet
+ teclado musical
+ vinil falsificado da volta do One Direction

Com isso, sua mochila terá o valor de 17900 dinheiros e peso de 14.1 unidades de massa.


3ª tentativa: 30 gerações:
Maior valor: 12550 | Peso: 11.6
Maior valor: 13000 | Peso: 11.5
Maior valor: 14050 | Peso: 13.6
Maior valor: 14500 | Peso: 13.5
Maior valor: 15400 | Peso: 12.1

Você deve pegar os se

Na última etapa do algoritmo definimos a função principal do código que fará a iteração das gerações e decorrente propagação do algoritmo na populações. Em cada iteração é pega a população atual e calcula-se seu fit, ou seja, o quão bem cada individuo esta indo no problema. Após o calculo do fit, é feito a escolha dos próximos individuos, os individuos selecionados terão a chance de passar pelo cruzamento ou mutação, ou até mesmo ambos. Estas etapas são necessárias para que haja uma variação entre a população inicial e a seguinte, deste modo, o problema pode convergir para uma solução.

## Conclusão

Neste notebook, nosso objetivo era sintetizar um algoritmo capaz de encontrar a melhor solução para um problema onde tem-se uma mochila com limite de peso (neste caso de 15 Kg) e deseja-se levar o maior valor monetário possível em itens que possuem valores e pesos variados. Para isso, foi criado um algoritmo parecido com o das caixas não binárias, onde o valor 1 representa a presença de tal objeto na mochila e o valor 0 representa sua ausência, deste modo, a implementação do código é dada de maneira simples, já que este é um problema ja resolvido em um notebook passado. Algo importante a se notar é que desta vez temos um algoritimo de maximização com uma limitação, já que queremos o maior valor monetário possível, tentamos maximizar este valor, enquanto nos mantemos abaixo do valor limite de 15 Kg para a mochila.

Para determinar a eficácia do algoritmo, este foi rodado 10 vezes com variações em seu número de gerações, estando este entre 10 e 100, em cada uma das iterações é possível ver a melhor solução encontrada com aquela quantidade de gerações. Separadamente não há uma conclusão a ser feita, mas quando comparamos os resultados obtidos podemos observar que conforme o aumento da quantidade de gerações o algoritmo _tende_ a encontrar uma melhor solução, mas este nem sempre será o caso.

## Playground



Outro método de calcular o valor e o peso da mochila

In [5]:
ind = [random.randint(0,1) for _ in range(NUM_OBJETOS)]
print(ind)

fit = sum([OBJETOS[i]['valor'] for i in [ORDEM_DOS_NOMES[i] for i, j in enumerate(ind) if j]])
peso = sum([OBJETOS[i]['peso'] for i in [ORDEM_DOS_NOMES[i] for i, j in enumerate(ind) if j]])

print(fit, peso)

[0, 1, 1, 0, 0, 1, 0, 0, 1, 0]
16000 30.5
